In [1]:
import numpy as np
from cvxopt import matrix, solvers

In [2]:
np.random.seed(22)
# simulated  samples
means = [[2, 2], [4, 2]]
cov = [[.3, .2], [.2, .3]]
N = 10
X0 = np.random.multivariate_normal(means[0], cov, N) # blue class data 
X1 = np.random.multivariate_normal(means[1], cov, N) # red class data
X = np.concatenate((X0, X1), axis = 0)               # all data 
y = np.concatenate((np.ones(N), -np.ones(N)), axis = 0) # label 

In [3]:
# solving the dual problem (variable: lambda)
V = np.concatenate((X0, -X1), axis = 0) # V in the book
Q = matrix(V.dot(V.T))
p = matrix(-np.ones((2*N, 1))) # objective function 1/2 lambda^T*Q*lambda - 1^T*lambda 
# build A, b, G, h 
G = matrix(-np.eye(2*N))
h = matrix(np.zeros((2*N, 1)))
A = matrix(y.reshape(1, -1)) 
b = matrix(np.zeros((1, 1))) 
solvers.options['show_progress'] = False
sol = solvers.qp(Q, p, G, h, A, b)

In [10]:
l = np.array(sol['x']) # solution lambda 

# calculate w
w = V.T.dot(l)
# calculate b 
S = np.where(l > 1e-5)[0] # support set, 1e-8 to avoid small value of l.
b = np.mean(y[S].reshape(-1, 1) - X[S,:].dot(w))
print('Number of suport vectors = ', S.size)
print('w = ', w.T)
print('b = ', b)

Number of suport vectors =  3
w =  [[-2.00984382  0.64068336]]
b =  4.668560683288087


In [9]:
# solution by sklearn 
from sklearn.svm import SVC

model = SVC(kernel = 'linear', C = 1e5) # just a big number 
model.fit(X, y) 

w = model.coef_
b = model.intercept_
print('w = ', w)
print('b = ', b)

w =  [[-2.00971102  0.64194082]]
b =  [4.66595309]
